In [49]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset
import pytorch_lightning as pl
import optuna

In [50]:
TAMANHO_TESTE = 0.1
TAMANHO_VALIDACAO = 0.1

SEMENTE_ALEATORIA = 14042003

NUM_EPOCAS = 60

In [51]:
dados = pickle.load(open('df_teste', 'rb'))
target =  ['Trg']
Features = list(dados.columns)[1:]

### Data Module

In [52]:
class DataModule(pl.LightningDataModule):
    def __init__(
        self,
        batch_size = 64,
        num_workers = 4,
    ):
        super().__init__()

        self.batch_size = batch_size
        self.num_workers = num_workers


    def setup(self, stage):
       
        features = list(dados.columns)[1:]
        target = ["Trg"]

        df = dados

        df = df.reindex(features + target, axis=1)

        indices = df.index
        indices_treino_val, indices_teste = train_test_split(
            indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
        )

        df_treino_val = df.loc[indices_treino_val]
        df_teste = df.loc[indices_teste]

        indices = df_treino_val.index
        indices_treino, indices_val = train_test_split(
            indices,
            test_size=TAMANHO_TESTE,
            random_state=SEMENTE_ALEATORIA,
        )

        df_treino = df.loc[indices_treino]
        df_val = df.loc[indices_val]

        X_treino = df_treino.reindex(features, axis=1).values
        y_treino = df_treino.reindex(target, axis=1).values

        self.x_scaler = MaxAbsScaler()
        self.x_scaler.fit(X_treino)

        self.y_scaler = MaxAbsScaler()
        self.y_scaler.fit(y_treino)

        if stage == "fit":
            X_val = df_val.reindex(features, axis=1).values
            y_val = df_val.reindex(target, axis=1).values

            X_treino = self.x_scaler.transform(X_treino)
            y_treino = self.y_scaler.transform(y_treino)

            X_val = self.x_scaler.transform(X_val)
            y_val = self.y_scaler.transform(y_val)

            self.X_treino = torch.tensor(X_treino, dtype=torch.float32)
            self.y_treino = torch.tensor(y_treino, dtype=torch.float32)

            self.X_val = torch.tensor(X_val, dtype=torch.float32)
            self.y_val = torch.tensor(y_val, dtype=torch.float32)

        if stage == "test":
            X_teste = df_teste.reindex(features, axis=1).values
            y_teste = df_teste.reindex(target, axis=1).values

            X_teste = self.x_scaler.transform(X_teste)
            y_teste = self.y_scaler.transform(y_teste)

            self.X_teste = torch.tensor(X_teste, dtype=torch.float32)
            self.y_teste = torch.tensor(y_teste, dtype=torch.float32)

    def train_dataloader(self):
        return DataLoader(
            TensorDataset(self.X_treino, self.y_treino),
            batch_size=self.batch_size,
            num_workers=self.num_workers,
        )

    def val_dataloader(self):
        return DataLoader(
            TensorDataset(self.X_val, self.y_val),
            batch_size=self.batch_size,
            num_workers=self.num_workers,
        )

    def test_dataloader(self):
        return DataLoader(
            TensorDataset(self.X_teste, self.y_teste),
            batch_size=self.batch_size,
            num_workers=self.num_workers,
        )

In [53]:
class MLP(pl.LightningModule):
    def __init__(
        self, num_dados_entrada, num_camadas, num_neuronios, ativacao, taxa_dropout, num_targets, lr
    ):
        super().__init__()

        self.lr = lr
        
        funcoes_de_ativacao= {
            'ReLU': nn.ReLU(),
            'Sigmoid': nn.Sigmoid(),
            'Tanh': nn.Tanh(),
        }
        
        camadas =[]
        
        for _ in range(num_camadas):
            
            camadas.append(nn.Linear(num_dados_entrada, num_neuronios))
            camadas.append(funcoes_de_ativacao[ativacao])
            camadas.append(nn.Dropout(taxa_dropout))
            num_dados_entrada = num_neuronios
            
        camadas.append(nn.Linear(num_neuronios, num_targets))
            
        self.camadas = nn.Sequential(*camadas)

        self.fun_perda = F.l1_loss

        self.perdas_treino = []
        self.perdas_val = []

        self.curva_aprendizado_treino = []
        self.curva_aprendizado_val = []

    def forward(self, x):
        x = self.camadas(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.fun_perda(y, y_pred)

        self.log("loss", loss, prog_bar=True)
        self.perdas_treino.append(loss)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.fun_perda(y, y_pred)

        self.log("val_loss", loss, prog_bar=True)
        self.perdas_val.append(loss)

        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.fun_perda(y, y_pred)

        self.log("test_loss", loss)

        return loss

    def on_train_epoch_end(self):
        # Atualiza curva de aprendizado
        perda_media = torch.stack(self.perdas_treino).mean()
        self.curva_aprendizado_treino.append(float(perda_media))
        self.perdas_treino.clear()

    def on_validation_epoch_end(self):
        # Atualiza curva de aprendizado
        perda_media = torch.stack(self.perdas_val).mean()
        self.curva_aprendizado_val.append(float(perda_media))
        self.perdas_val.clear()

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr = self.lr)
        return optimizer

In [54]:
def funcao_objetivo(trial):
    
    num_camadas = trial.suggest_int('num_camadas', 1 , 5)
    num_neuronios = trial.suggest_int('num_neuronios', 4, 20)
    ativacao = trial.suggest_categorical('ativacao', ['ReLU', 'Tanh', 'Sigmoid'])
    taxa_dropout =  trial.suggest_categorical('taxa_dropout', [0, 0.1, 0.05, 0.15])
    lr = trial.suggest_categorical('lr', [1e-5, 1e-4, 1e-3, 1e-2])
    
    treinador = pl.Trainer(max_epochs=NUM_EPOCAS)
    dm = DataModule()

    minha_mlp = MLP(20, num_camadas, num_neuronios, ativacao, taxa_dropout, 1, lr
)    
    treinador.fit(minha_mlp, dm)
    
    minha_mlp.eval()
    dm.setup("test")

    with torch.no_grad():
        X_true = dm.X_teste

        y_true = dm.y_teste
        y_true = dm.y_scaler.inverse_transform(y_true)

        y_pred = minha_mlp(X_true)
        print(y_pred[0])
        y_pred = dm.y_scaler.inverse_transform(y_pred)
        print(y_pred[0])

        RMSE = mean_squared_error(y_true, y_pred, squared = False)
        
    return RMSE

In [55]:

study = optuna.create_study(direction = 'minimize')
study.optimize(funcao_objetivo, n_trials = 100)

[I 2024-05-18 17:40:28,583] A new study created in memory with name: no-name-1f568043-92d2-4e60-890c-b8b40056e69f
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type       | Params
---------------------------------------
0 | camadas | Sequential | 639   
---------------------------------------
639       Trainable params
0         Non-trainable params
639       Total params
0.003     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

c:\venv\ilumpy\lib\site-packages\pytorch_lightning\trainer\trainer.py:1558: PossibleUserWarning: The number of training batches (8) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

c:\venv\ilumpy\lib\site-packages\pytorch_lightning\callbacks\progress\base.py:249: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

[W 2024-05-19 07:33:39,894] Trial 91 failed because of the following error: PermissionError(13, 'O arquivo já está sendo usado por outro processo')
Traceback (most recent call last):
  File "c:\venv\ilumpy\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\davi23037\AppData\Local\Temp\ipykernel_30420\3624434121.py", line 14, in funcao_objetivo
    treinador.fit(minha_mlp, dm)
  File "c:\venv\ilumpy\lib\site-packages\pytorch_lightning\trainer\trainer.py", line 582, in fit
    call._call_and_handle_interrupt(
  File "c:\venv\ilumpy\lib\site-packages\pytorch_lightning\trainer\call.py", line 38, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
  File "c:\venv\ilumpy\lib\site-packages\pytorch_lightning\trainer\trainer.py", line 624, in _fit_impl
    self._run(model, ckpt_path=self.ckpt_path)
  File "c:\venv\ilumpy\lib\site-packages\pytorch_lightning\trainer\trainer.py", line 1061, in _run
    results =

PermissionError: [WinError 32] O arquivo já está sendo usado por outro processo: 'C:/Users/davi23037/OneDrive - ILUM ESCOLA DE CIÊNCIA/Redes Neurais e Algorítmos Genéticos/Trabalho final/lightning_logs/version_108/checkpoints/epoch=53-step=432.ckpt'

#### Best trial


In [56]:
best = study.best_trial

In [71]:
print(best.params)

{'num_camadas': 3, 'num_neuronios': 12, 'ativacao': 'ReLU', 'taxa_dropout': 0, 'lr': 0.01}


### Fewer parameters

In [86]:
fewer = study.trials[17]

In [87]:
print(fewer.params)

{'num_camadas': 2, 'num_neuronios': 4, 'ativacao': 'ReLU', 'taxa_dropout': 0.05, 'lr': 0.01}
